In [14]:
import numpy as np
import pandas as pd
import re
import json
import time
import os

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from collections import Counter

import datetime
from dateutil.parser import parse

C:\Users\USER\AppData\Local\Temp\ipykernel_22400\1706855644.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
path_ = r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\all_2"

In [4]:
file_list = os.listdir(path_)
file_list

['2012_demand.txt',
 '2012_supply.txt',
 '2013_demand.txt',
 '2013_supply.txt',
 '2014_demand.txt',
 '2014_supply.txt']

In [8]:
test_ = file_list[1]
df_ = pd.read_csv(path_+ "\\" + test_,sep = '\t')
df_.head()

,Unnamed: 0,일자,분류,물품명,수량,단가,금액,label,bank,year
0,0,NaN,분류,물품명,실수량,단가,금액,가공식품 / 반찬,2012_광역,2012
1,1,20120104.0,과자류,초코과자개,30679,2708.91,83106750,과자류,2012_광역,2012
2,2,20120106.0,과자류,초코과자개,7541,1632.97,12314250,과자류,2012_광역,2012
3,3,20120120.0,기타가공식품,기타 부식류개,17747,1429.72,25373190,가공식품 / 반찬,2012_광역,2012
4,4,20120202.0,기타가공식품,기타 식재료,196,5054.12,990608,가공식품 / 반찬,2012_광역,2012


In [43]:
tar_date = parse(str(int(df_["일자"][1])))
datetime.datetime.strftime(tar_date,"%Y-%m-%d")

'2012-01-04'

In [46]:

test_ = df_["일자"][1]
def trans_datetime(test_):
    try:
        tar_date = parse(str(int(test_)))
        out_ = datetime.datetime.strftime(tar_date,"%Y-%m-%d")
    except:
        out_ = "NA"
    return out_

In [49]:
days = [trans_datetime(col["일자"]) for row,col in df_.iterrows()]

In [52]:
df_["date"] = df_.일자.map(lambda x: trans_datetime(x))
df_

,Unnamed: 0,일자,분류,물품명,수량,단가,금액,label,bank,year,date
0,0,NaN,분류,물품명,실수량,단가,금액,가공식품 / 반찬,2012_광역,2012,NA
1,1,20120104.0,과자류,초코과자개,30679,2708.91,83106750,과자류,2012_광역,2012,2012-01-04
2,2,20120106.0,과자류,초코과자개,7541,1632.97,12314250,과자류,2012_광역,2012,2012-01-06
3,3,20120120.0,기타가공식품,기타 부식류개,17747,1429.72,25373190,가공식품 / 반찬,2012_광역,2012,2012-01-20
4,4,20120202.0,기타가공식품,기타 식재료,196,5054.12,990608,가공식품 / 반찬,2012_광역,2012,2012-02-02
...,...,...,...,...,...,...,...,...,...,...,...
144598,1263,20121126.0,대용식(빵),빵개,15,940,14100,대용식(빵),2012년 화성시행복나눔푸드뱅크 접수 및 제공,2012,2012-11-26
144599,1264,20121128.0,대용식(빵),빵개,12,1366.67,16400,대용식(빵),2012년 화성시행복나눔푸드뱅크 접수 및 제공,2012,2012-11-28
144600,1265,20121207.0,과자류,초코과자개,1020,5394.12,5502000,과자류,2012년 화성시행복나눔푸드뱅크 접수 및 제공,2012,2012-12-07
144601,1266,20121231.0,대용식(빵),빵개,1676,100,167600,대용식(빵),2012년 화성시행복나눔푸드뱅크 접수 및 제공,2012,2012-12-31


In [57]:
df_ = df_[df_["date"]!="NA"]
df_["수량"] = df_.수량.map(lambda x: int(x))
df_ = df_[["date","bank","label","수량"]]
df_out = df_.groupby(by=["date","bank","label"]).sum()
df_out.to_excel(r"C:\Users\USER\Desktop\All\Project\23_VRP\Meeting\test.xlsx")
df_out

수량
date       bank                      label          
2012-01-01 2012년 광주시참사랑푸드뱅크 접수 및 제공  대용식(빵)       78
           2012년 기쁨가득한푸드뱅크 접수 및 제공   대용식(빵)      206
           2012년 서안성푸드뱅크 접수 및 제공     대용식(빵)      111
           2012년 성남시푸드뱅크 접수 및 제공     대용식(빵)     8497
           2012년 수원시권선푸드뱅크 접수 및 제공   대용식(빵)       29
...                                              ...
2012-12-31 2012년 포천시푸드뱅크 접수 및 제공     가공식품 / 반찬    40
                                     대용식(빵)       73
                                     즉석/편의식품      25
           2012년 화성시나래울푸드마켓 접수 및 제공  대용식(빵)       35
           2012년 화성시행복나눔푸드뱅크 접수 및 제공 대용식(빵)     1843

[27921 rows x 1 columns]

In [58]:
df_out = df_.groupby(by=["bank","date","label"]).sum()
df_out.to_excel(r"C:\Users\USER\Desktop\All\Project\23_VRP\Meeting\test2.xlsx")
df_out

수량
bank                      date       label           
2012_광역                   2012-01-04 과자류        30679
                          2012-01-06 과자류         7541
                          2012-01-20 가공식품 / 반찬  17747
                          2012-02-02 가공식품 / 반찬    196
                          2012-02-03 가공식품 / 반찬   3600
...                                               ...
2012년 화성시행복나눔푸드뱅크 접수 및 제공 2012-11-23 대용식(빵)       277
                          2012-11-26 대용식(빵)       172
                          2012-11-28 대용식(빵)        12
                          2012-12-07 과자류         1020
                          2012-12-31 대용식(빵)      1843

[27921 rows x 1 columns]